In [2]:
from google.colab import drive
drive.mount('/content/gdrive/')
import sys
sys.path.append('/content/gdrive/MyDrive/Implementasi/6. Transformer')
%cd '/content/gdrive/MyDrive/Implementasi/6. Transformer'
!pwd
print(sys.version)

Mounted at /content/gdrive/
/content/gdrive/MyDrive/Implementasi/6. Transformer
/content/gdrive/MyDrive/Implementasi/6. Transformer
3.7.15 (default, Oct 12 2022, 19:14:55) 
[GCC 7.5.0]


In [3]:
from transformer.layers import make_model
from transformer.train import run_epoch, data_gen, \
        TrainState, SimpleLossCompute, LabelSmoothing
from transformer.utils import rate_warmup, rate_decay
import torch, os
from torch.optim import Adam
from torch.optim.lr_scheduler import LambdaLR
from datetime import datetime

In [3]:
assert torch.cuda.is_available()

In [4]:
embedding_data = torch.load('../5. Embedding/embedding_base.h5')
embedding_data

KeyboardInterrupt: ignored

In [ ]:
embedding_data['MAD']['embedding'][512]['state']['in_embed.weight'].shape

In [ ]:
embedding_data['TEKS'].shape

In [ ]:
BATCH_SIZE, EPOCH, EMBEDDING_SIZE, PAD_TOKEN, DEVICE = 2048, 200, 512, 2, 'cuda'
N, D_FF, HEAD, DROPOUT, LABEL_SMOOTHING, LR_MODE = 6, 2048, 8, 0.1, 0.1, 'warmup'
SRC_VOCAB_LEN = embedding_data['MAD']['embedding'][512]['state']['in_embed.weight'].shape[0]
DST_VOCAB_LEN = embedding_data['IND']['embedding'][512]['state']['in_embed.weight'].shape[0]

In [4]:
def run_training(embedding_file, model_file, start_epoch=0):
    embedding_data = torch.load(embedding_file, map_location=torch.device(DEVICE))
    SRC_VOCAB_LEN = embedding_data['MAD']['embedding'][EMBEDDING_SIZE]['state']['in_embed.weight'].shape[0]
    DST_VOCAB_LEN = embedding_data['IND']['embedding'][EMBEDDING_SIZE]['state']['in_embed.weight'].shape[0]
    model = make_model(
        SRC_VOCAB_LEN,
        DST_VOCAB_LEN,
        src_vocab=embedding_data['MAD']['embedding'][EMBEDDING_SIZE]['state']['in_embed.weight'],
        dst_vocab=embedding_data['IND']['embedding'][EMBEDDING_SIZE]['state']['in_embed.weight'],
        N=N, d_ff=D_FF, h=HEAD, dropout=DROPOUT,
        d_model=EMBEDDING_SIZE,
        device=DEVICE,
    )
    data_iter = list(data_gen(torch.from_numpy(embedding_data['TEKS']).to(DEVICE), BATCH_SIZE))
    train_state = TrainState()
    optimizer = Adam(model.parameters(), lr=1, weight_decay=1e-5, betas=(0.9, 0.98))
    criterion = LabelSmoothing(size=DST_VOCAB_LEN, padding_idx=PAD_TOKEN, smoothing=LABEL_SMOOTHING)

    if LR_MODE == 'warmup':
      lr_scheduler = LambdaLR(
          optimizer=optimizer,
          lr_lambda=lambda step: rate_warmup(
              step, model_size=EMBEDDING_SIZE, warmup=400
          ),
      )
    elif LR_MODE == 'decay':
      lr_scheduler = LambdaLR(
          optimizer=optimizer,
          lr_lambda=lambda step: rate_decay(
              step, decay=0.001, init=2
          ),
      )

    if os.path.exists(model_file):
        data = torch.load(model_file)
        model.load_state_dict(data['model_state'])
        optimizer.load_state_dict(data['optimizer_state'])
        lr_scheduler.load_state_dict(data['lr_scheduler_state'])
        train_state.load_state_dict(data['train_state'])
        print('Model loaded successfully!')

    with open(model_file + '.log', 'a', encoding='utf-8') as log_file:
      for epoch in range(start_epoch, EPOCH):
          try: 
              loss = SimpleLossCompute(model.generator, criterion)
              loss_val, train_state = run_epoch(data_iter, model, loss, optimizer, lr_scheduler, 
                                         train_state=train_state, log_file=log_file)
              log_file.write("Progress Epoch: %5d , Loss %3f \n" % (epoch, loss_val))
              print("Progress Epoch: %5d , Loss %3f (%s)" % (epoch, loss_val, datetime.now().isoformat()))
          except KeyboardInterrupt:
              print('\nTraining interrupted')
              break

    torch.save({
        'model_state': model.state_dict(),
        'optimizer_state': optimizer.state_dict(),
        'lr_scheduler_state': lr_scheduler.state_dict(),
        'train_state': train_state.state_dict(),
    }, model_file)
    print('Training state saved')



In [ ]:
BATCH_SIZE, EPOCH, EMBEDDING_SIZE, PAD_TOKEN, DEVICE = 200, 1000, 512, 2, 'cuda'
N, D_FF, HEAD, DROPOUT, LABEL_SMOOTHING, LR_MODE = 6, 2048, 8, 0.1, 0.1, 'warmup'
run_training('../5. Embedding/embedding_base.h5', 'model_high_base.h5', 950)

In [ ]:
BATCH_SIZE, EPOCH, EMBEDDING_SIZE, PAD_TOKEN, DEVICE = 200, 1000, 512, 2, 'cuda'
N, D_FF, HEAD, DROPOUT, LABEL_SMOOTHING, LR_MODE = 6, 2048, 8, 0.1, 0.1, 'warmup'
run_training('../5. Embedding/embedding_wp8k.h5', 'model_high_wp8k.h5', 10)

Streaming output truncated to the last 5000 lines.
Epoch Step:    127 | Accumulation Step: 128 | Loss:   0.14 | Tokens / Sec:  2095.6 | Learning Rate: 1.8e-04
Epoch Step:    128 | Accumulation Step: 129 | Loss:   0.15 | Tokens / Sec:  2137.3 | Learning Rate: 1.8e-04
Epoch Step:    129 | Accumulation Step: 130 | Loss:   0.11 | Tokens / Sec:  2061.8 | Learning Rate: 1.8e-04
Epoch Step:    130 | Accumulation Step: 131 | Loss:   0.14 | Tokens / Sec:  2273.2 | Learning Rate: 1.8e-04
Epoch Step:    131 | Accumulation Step: 132 | Loss:   0.12 | Tokens / Sec:  2072.3 | Learning Rate: 1.8e-04
Epoch Step:    132 | Accumulation Step: 133 | Loss:   0.17 | Tokens / Sec:  2182.9 | Learning Rate: 1.8e-04
Epoch Step:    133 | Accumulation Step: 134 | Loss:   0.15 | Tokens / Sec:  2050.3 | Learning Rate: 1.8e-04
Epoch Step:    134 | Accumulation Step: 135 | Loss:   0.17 | Tokens / Sec:  2112.6 | Learning Rate: 1.8e-04
Epoch Step:    135 | Accumulation Step: 136 | Loss:   0.13 | Tokens / Sec:  2138.6 | 

In [ ]:
BATCH_SIZE, EPOCH, EMBEDDING_SIZE, PAD_TOKEN, DEVICE = 200, 500, 256, 2, 'cuda'
N, D_FF, HEAD, DROPOUT, LABEL_SMOOTHING, LR_MODE = 6, 2048, 2, 0.3, 0.1, 'decay'
run_training('../5. Embedding/embedding_base.h5', 'model_low_base.h5')

In [ ]:
BATCH_SIZE, EPOCH, EMBEDDING_SIZE, PAD_TOKEN, DEVICE = 200, 500, 256, 2, 'cuda'
N, D_FF, HEAD, DROPOUT, LABEL_SMOOTHING, LR_MODE = 6, 2048, 2, 0.3, 0.1, 'decay'
run_training('../5. Embedding/embedding_wp8k.h5', 'model_low_wp8k.h5')